In [16]:
import requests

In [21]:
# The base URL to use for the request
base_url = "https://api.carbonintensity.org.uk/regional/intensity/"
from_date = '2024-08-26T00:00Z'
to_date = '2024-08-26T04:00Z'
postcode = "BN3"


In [25]:
# Make the GET request
response = requests.get(base_url+from_date+"/"+to_date+"/postcode/"+postcode)

# Check if the request was successful
if response.status_code == 200:
    print("Response received: OK")
else:
    print(f"Error: {response.status_code}")

Response received: OK


In [24]:
print(response.json())

{'data': {'regionid': 14, 'shortname': 'South East England', 'postcode': 'BN3', 'data': [{'from': '2024-08-25T23:30Z', 'to': '2024-08-26T00:00Z', 'intensity': {'forecast': 142, 'index': 'moderate'}, 'generationmix': [{'fuel': 'biomass', 'perc': 0.4}, {'fuel': 'coal', 'perc': 0}, {'fuel': 'imports', 'perc': 0.2}, {'fuel': 'gas', 'perc': 36}, {'fuel': 'nuclear', 'perc': 7}, {'fuel': 'other', 'perc': 0}, {'fuel': 'hydro', 'perc': 2.4}, {'fuel': 'solar', 'perc': 0}, {'fuel': 'wind', 'perc': 54}]}, {'from': '2024-08-26T00:00Z', 'to': '2024-08-26T00:30Z', 'intensity': {'forecast': 139, 'index': 'moderate'}, 'generationmix': [{'fuel': 'biomass', 'perc': 0.4}, {'fuel': 'coal', 'perc': 0}, {'fuel': 'imports', 'perc': 0.1}, {'fuel': 'gas', 'perc': 35.1}, {'fuel': 'nuclear', 'perc': 7}, {'fuel': 'other', 'perc': 0}, {'fuel': 'hydro', 'perc': 2.4}, {'fuel': 'solar', 'perc': 0.1}, {'fuel': 'wind', 'perc': 54.9}]}, {'from': '2024-08-26T00:30Z', 'to': '2024-08-26T01:00Z', 'intensity': {'forecast': 13